In [1]:
from clients.exchanges import KucoinFutures
from pandas import Timestamp, Timedelta

SYMBOL = 'XBTUSDTM'

TIMEFRAME = '1m'

SINCE = Timestamp.utcnow() - Timedelta(hours=6)

client = KucoinFutures()

In [2]:
order_book_data = client.fetch_order_book(SYMBOL)

order_book_data

,bid_price,bid_quantity,ask_price,ask_quantity,symbol,timestamp,nonce
0,61891.2,107.0,61891.3,764.0,XBTUSDTM,2024-08-27 14:21:38.517,1706099483416
1,61881.9,2.0,61893.7,298.0,XBTUSDTM,2024-08-27 14:21:38.517,1706099483416
2,61881.5,80.0,61893.8,50.0,XBTUSDTM,2024-08-27 14:21:38.517,1706099483416
3,61881.4,72.0,61894.4,440.0,XBTUSDTM,2024-08-27 14:21:38.517,1706099483416
4,61880.3,386.0,61895.5,68.0,XBTUSDTM,2024-08-27 14:21:38.517,1706099483416
...,...,...,...,...,...,...,...
995,60902.5,1.0,63230.5,3.0,XBTUSDTM,2024-08-27 14:21:38.517,1706099483416
996,60901.0,184.0,63231.1,1.0,XBTUSDTM,2024-08-27 14:21:38.517,1706099483416
997,60900.0,779.0,63232.4,4.0,XBTUSDTM,2024-08-27 14:21:38.517,1706099483416
998,60899.1,6.0,63235.0,1.0,XBTUSDTM,2024-08-27 14:21:38.517,1706099483416


In [2]:
ohlcv_data = client.fetch_ohlcv(SYMBOL, TIMEFRAME, SINCE)

ohlcv_data.tail()

,open,high,low,close,volume
timestamp,,,,,
2024-08-27 11:46:00,62492.1,62492.1,62441.1,62453.5,8064.0
2024-08-27 11:47:00,62462.5,62462.5,62431.7,62432.6,1768.0
2024-08-27 11:48:00,62432.6,62457.2,62432.6,62438.2,4534.0
2024-08-27 11:49:00,62434.5,62466.9,62434.5,62466.8,4857.0
2024-08-27 11:50:00,62466.8,62466.8,62438.4,62438.4,3938.0


In [8]:
import plotly.graph_objects as go

# Create a candlestick chart
fig = go.Figure()

# Add candlestick trace
fig.add_trace(go.Candlestick(
    x=ohlcv_data.index,
    open=ohlcv_data['open'],
    high=ohlcv_data['high'],
    low=ohlcv_data['low'],
    close=ohlcv_data['close'],
    name='Candlestick'
))

order_book_data['x'] = ohlcv_data.index.max()

# Add ask orders as smaller, semi-transparent bubbles
fig.add_trace(go.Scatter(
    x=order_book_data['x'],
    y=order_book_data['ask_price'],
    mode='markers',
    marker=dict(
        size=order_book_data['ask_quantity'] * 0.001,  # Further reduced size multiplier
        color='rgba(255, 0, 0, 0.3)',  # More transparent red for ask orders
        line=dict(width=0.5, color='rgba(255, 0, 0, 0.6)')
    ),
    name='Ask Orders',
    text=order_book_data['ask_quantity'],  # Display quantity on hover
    hoverinfo='text'
))

# Add bid orders as smaller, semi-transparent bubbles
fig.add_trace(go.Scatter(
    x=order_book_data['x'],
    y=order_book_data['bid_price'],
    mode='markers',
    marker=dict(
        size=order_book_data['bid_quantity'] * 0.001,  # Further reduced size multiplier
        color='rgba(0, 255, 0, 0.3)',  # More transparent green for bid orders
        line=dict(width=0.5, color='rgba(0, 255, 0, 0.6)')
    ),
    name='Bid Orders',
    text=order_book_data['bid_quantity'],  # Display quantity on hover
    hoverinfo='text'
))

# Update layout
fig.update_layout(
    xaxis_title='T',
    yaxis_title='USDT',
    xaxis_rangeslider_visible=False
)

# Show the plot
fig.show()